In [ ]:
import gradio as gr
import os
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
import re

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

os.environ["OPENAI_API_KEY"] = 'YOUR_API_KEY'
llm = ChatOpenAI(model_name='gpt-3.5-turbo-0613', temperature=0)

template = """

{instructions}:

{content}
"""
index = 0
texts = [
"""
Once upon a time, in a quaint little village, two best friends named Lily and Emily embarked on a leisurely walk in the enchanting forest. As they strolled along the winding path, the golden hues of the setting sun filtered through the foliage, casting a warm glow upon their faces. It was a serene moment, as if nature itself was bidding farewell to the day.

With every step they took, the friends immersed themselves in the soothing symphony of rustling leaves and melodious bird songs. The air was crisp and filled with the scent of pine, evoking a sense of tranquility in their hearts. They reveled in the magic of the forest, their laughter echoing amidst the ancient trees.

Lily and Emily paused near a babbling brook, its crystal-clear waters meandering through moss-covered rocks. They knelt down, cupping their hands, and took sips of the cool, refreshing stream. The sound of their laughter blended harmoniously with the babbling water, creating a delightful melody that danced through the woods.

As the sun's rays grew softer and began to dip below the horizon, the friends found a cozy spot beneath a majestic oak tree. They settled on a patchwork blanket, their fingers intertwining as they leaned back against the sturdy trunk. A serene silence enveloped them, broken only by the occasional whisper of a gentle breeze.

In the fading light, Lily and Emily shared stories and dreams, their voices laced with nostalgia and hope. They watched as the sky transformed into a canvas painted with hues of orange, pink, and purple. Hand in hand, they embraced the beauty of the moment, cherishing their unbreakable bond and the memories they were creating.

As the stars began to twinkle above, Lily and Emily rose from their peaceful sanctuary, their hearts filled with gratitude for the precious time they had spent in nature's embrace. With their spirits uplifted and their friendship strengthened, they made their way back home, carrying the magic of the forest and the last rays of the sun within them forever.
"""]


system_message_prompt = SystemMessagePromptTemplate.from_template(template)
full_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
chain = LLMChain(llm=llm, prompt=full_prompt)

# Functions to handle button click events
def change(prompt):
    
    global index 
    
    res = chain.run(instructions='Rewrite the following story but '+prompt,content=texts[index])
        
    index =0
    texts[index]=res
    return res
    
def zoom_in():
    
    global index
    if index>0:
        index-=1
    return texts[index]

def zoom_out():
    
    
    global index
    if index==len(texts)-1:

        content = texts[index]
        
        inst = 'Summarize the following text:'
        
        if index==0:
            # star by summarizing the paragraphs
            contet = ''
            ps = texts[index].split('\n\n')
            
            for i in range(len(ps)):
                content+=f'Paragraph {i+1}\n'
                content+=ps[i]
                content+='\n\n'


            inst = """
Summarize the paragraphs below.
Make sure to format the output with "Paragraph NUMBER:"
            """
    
        res = chain.run(instructions=inst,content=content)
                
        res = re.sub(r'Paragraph \d+:\s+', '\n', res)
        
        texts.append(res)
    
    index +=1 
    
    return texts[index]


with gr.Blocks() as demo:
    with gr.Row():
        output = gr.Textbox(label="Text", lines=15, value=texts[index])

    with gr.Row():
        prompt = gr.Textbox(label="Prompt")
        change_btn = gr.Button("Apply")

        zoomin_btn = gr.Button("Zoom in")
        zoomout_btn = gr.Button("Zoom out")

        change_btn.click(fn=change, inputs=prompt, outputs=output, api_name="Change")
        zoomin_btn.click(fn=zoom_in, outputs=output, api_name="Zoom in")
        zoomout_btn.click(fn=zoom_out, outputs=output, api_name="Zoom out")


demo.launch(inbrowser=True)

